# `Python to MySQL: Hiccups you will encounter`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

# `Outcome:`

+ `Problems with Date-Time`
+ `Handling NaN`
+ `Schema Considerations`
+ `Dot Notation Naming`

In [12]:
import pandas as pd # dataframe stuff
# import json # 
from collections import defaultdict # dictionary where values are a list
import numpy as np # formatting data with Nan
from datetime import date
import datetime

# import collections # This is used to help find duplicate values later

In [ ]:
# Setup MySQL for us to send data and create a new DB

import mysql.connector as mysql # establish connection to mysql 

from mysql.connector import MySQLConnection # using Class for function call

import pandas as pd  # Create Dataframe

from python_to_myql_config import read_db_config # calling config files, (you need to make this) 
# stored user credentials I have, or use the block of code below

import csv

In [ ]:
# I had to turn on mysql server for this to work next: on Mac terminal


# mysql.server start

# `If you want user` <font color=red>credential</font> `setup` <font color=red>go here</font>`:`

https://github.com/MrFuguDataScience/SendDataFrom-R-Python-to-SQL-PSQL-Mongo/blob/master/Python_to_mysql_configFiles.ipynb

**IF YOU DO NOT HAVE CREDENTIALS OR IT DOESN'T MATTER TO YOU TRY THIS:**

`
cnx = mysql.connector.connect(user='bambam', password='password',
                              host='127.0.0.1',
                              database='cowboy_doodah')
cnx.close()
`

+ Remember: replace, user, password and database with your information!

In [ ]:
db_config = read_db_config() # user credentials are mine, you will need to change this line

conn = MySQLConnection(**read_db_config()) # establish connection

mycursor = conn.cursor() # cursor, allows us to do queries,send/retrieve data

In [ ]:
# Show What DB we are using:

# show what database we are currently using:
mycursor.execute("SELECT DATABASE()")
mycursor.fetchall()

In [ ]:
# Turn this on, if you have issues sending data

mycursor.execute('SET GLOBAL local_infile=1')

# This checks if you have issues with exporting later due to security:
mycursor.execute('SHOW VARIABLES LIKE "secure_file_priv"')
mycursor.fetchall()

In [ ]:
# Check Db Using:

In [ ]:
# Check Tables:

# `Ex.1) Evaluating Dates/Time Formatting MySQL`

In [7]:
# Date-Time csv:
dates_df=pd.read_csv('date_time_ex.csv')
dates_df.head()

,createDate_date,dateScanned_date,finishedDate_date,modifyDate_date,pointsAwardedDate_date,purchaseDate_date
0,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12
1,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
2,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
3,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12
4,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,1.609632e+12


In [10]:
# Creating proper format for dates
divide_by_thous_dts=dates_df.div(1000, axis=0)

# doing this across all date columns
convert_to_dts=divide_by_thou_dts.apply(pd.to_datetime, errors='coerce',utc=True,unit='s')

# If you need date with time unblock the , %H:%M:%S")) portion of code and it will work BUT, CHANGE SCHEMA 
# to DATETIME instead of DATE!

pp_new=convert_to_dts.apply(lambda x: x.dt.strftime("%Y-%m-%d"))
                                        
#                                        , %H:%M:%S"))
pp_new.head()

,createDate_date,dateScanned_date,finishedDate_date,modifyDate_date,pointsAwardedDate_date,purchaseDate_date
0,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03
1,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-02
2,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-02
3,2021-01-03,2021-01-03,NaN,2021-01-03,NaN,2021-01-03
4,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03


# `Here is a potential issue:` 
+ **`['finishedDate_date','dateScanned_date','modifyDate_date']`** you have `NaN` for dates 
and empty string is not working
+ *let's investigate all the dates first to make sure that this is not a fluke and needs further testing*


`What is the problem with NaN values for MySQL import/Insert?`

In [13]:
pp_new.isnull().sum()

createDate_date              0
dateScanned_date             0
finishedDate_date         1411
modifyDate_date              0
pointsAwardedDate_date    1301
purchaseDate_date          458
dtype: int64

In [ ]:
# Create Schema:

# `Send Data To MySQL from Python`

In [ ]:
# Send Data: W/O Converting Dates Correctly (2 ways to do this)

# 1) 
"""
We have the option to grab data from one consistant column and use that as our source to work from to fill
NaN values. In this case I was considering using ['createDate_date']
"""




# 2)

"""
This was a dummy date so MySQL would take import and you can query this to find your Nan values easy
if you want I would think about replacing with CreateDate for consistency. This will depend on what you are
needing as an end result or personal preference.
"""
new_date_nan_changed=pp_new.replace(np.nan,'1900-01-01',regex=True)
new_date_nan_changed.head()

In [ ]:
# Send Dates: With Correct formatting for schema:



# `Ex.2) Other Errors That May Occur:`

+ Dot Notation
+ OID

In [25]:
# Data csv
df_parse=pd.read_csv('mysql_play_data.csv')

ex_df_subset=df_parse.loc[:,['_id.oid','bonusPointsEarned','rewardsReceiptItemList_pointsEarned',"needsFetchReview",
'competitiveProduct','deleted','userFlaggedQuantity']]
# .isnull().sum()

ex_df_subset.head()

,_id.oid,bonusPointsEarned,rewardsReceiptItemList_pointsEarned,needsFetchReview,competitiveProduct,deleted,userFlaggedQuantity
0,5ff1e1eb0a720f0523000575,500.0,NaN,False,NaN,NaN,5.0
1,5ff1e1bb0a720f052300056b,150.0,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,150.0,NaN,True,NaN,NaN,1.0
3,5ff1e1f10a720f052300057a,5.0,NaN,False,NaN,NaN,3.0
4,5ff1e1ee0a7214ada100056f,5.0,NaN,False,NaN,NaN,4.0


In [27]:
ex_df_subset.isnull().sum()

_id.oid                                   0
bonusPointsEarned                      1401
rewardsReceiptItemList_pointsEarned    6454
needsFetchReview                       6568
competitiveProduct                     6736
deleted                                7372
userFlaggedQuantity                    7082
dtype: int64

In [29]:
ex_df_subset.dtypes

_id.oid                                 object
bonusPointsEarned                      float64
rewardsReceiptItemList_pointsEarned    float64
needsFetchReview                        object
competitiveProduct                      object
deleted                                 object
userFlaggedQuantity                    float64
dtype: object

In [ ]:
# 2A.) Dot Notation from JSON data, explain and fix formatting

In [ ]:
# This is MySQL problem for importing so I changed dot notation of columns
new_subset_play.columns=new_subset_play.columns.str.replace('.','_')


In [ ]:
# 2B.) # OID issues explained


In [ ]:
# OID issue and convert:
# Changed this name _id.OID because of convention naming systems for DB and I don't want confusion or errors
new_subset_play=new_subset_play.rename(columns={new_subset_play.columns[7]:"OID"})

# Like, Share & <font color=red>SUB</font>scribe

# `Citations & Help:`

# ◔̯◔

https://stackoverflow.com/questions/24942538/sql-wont-insert-null-values-with-bulk-insert

https://stackoverflow.com/questions/45756557/inserting-nan-value-into-mysql-database

https://www.tutorialspoint.com/can-mysql-automatically-convert-empty-strings-to-null

https://stackoverflow.com/questions/41027882/convert-nan-values-to-null-in-mysql-columns-with-python-code

https://dev.mysql.com/doc/refman/8.0/en/problems-with-null.html

https://www.tutorialspoint.com/can-mysql-automatically-convert-empty-strings-to-null

https://www.programiz.com/python-programming/datetime/strftime

https://stackoverflow.com/questions/66477410/how-can-i-insert-id-oid-5ec796a7868d607e00f4b18d-and-time-15901

https://stackoverflow.com/questions/59817317/how-to-convert-multiple-columns-datatype-format-to-another-datatype-formats-at-a